Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from azure.core.credentials import AzureKeyCredential
from msrest.authentication import CognitiveServicesCredentials
import cv2
import pytesseract
import numpy as np
from jiwer import wer, cer
from array import array
from Levenshtein import distance as levenshtein_distance, hamming
from dotenv import dotenv_values
import time

In [3]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
config = dotenv_values(".env")
subscription_key = config["VISION_KEY"]
endpoint = config["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [4]:
import cv2
import numpy as np


def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


 

def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


In [5]:
image_path = "test2.jpeg"
image = cv2.imread(image_path)
image_gray = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
image_thresh = thresholding(image_gray)


In [ ]:

def ocr_tesseract(image):
    return pytesseract.image_to_string(image)

def ocr_azure(image_path):

    with open(image_path, 'rb') as image_file:
        read_response = computervision_client.read_in_stream(
        image=image_file,
        mode="Printed",
        raw=True
        )

        operation_id = read_response.headers['Operation-Location'].split('/')[-1]
        while True:
            read_result = computervision_client.get_read_result(operation_id)
            if read_result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        result = []
        if read_result.status == OperationStatusCodes.succeeded:
            for text_result in read_result.analyze_result.read_results:
                for line in text_result.lines:
                    print(line.text)
                    result.append(line.text)
    
    return " ".join([line for line in result])

def evaluate_recognition(ground_truth, recognized_text):
    lev_dist = levenshtein_distance(ground_truth, recognized_text)
    word_err = wer(ground_truth, recognized_text)
    char_err = cer(ground_truth, recognized_text)


    return {
        "Levenshtein Distance": lev_dist,
        "Word Error Rate": word_err,
        "Character Error Rate": char_err
    }


In [ ]:




ground_truth_text = ["Succes in rezolvarea temelor la laboratoarele de inteligenta artificiala"]

# OCR using Tesseract
tesseract_result = ocr_tesseract(image_thresh)
print("Tesseract OCR Result:\n", tesseract_result)

# OCR using Azure (replace with your credentials)
azure_result = ocr_azure(image_path)
print("Azure OCR Result:\n", azure_result)

# Evaluation
print("Tesseract Evaluation:", evaluate_recognition(ground_truth_text, tesseract_result))
print("Azure Evaluation:", evaluate_recognition(ground_truth_text, azure_result))


Tesseract OCR Result:
 PEMELOR Sa.

LAOORA toarele  ohe


Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!
Azure OCR Result:
 Lucces in resolvarea TEMELOR la LABORA toarele de Inteligenta Artificialà!
Tesseract Evaluation: {'Levenshtein Distance': 34, 'Word Error Rate': 1.0, 'Character Error Rate': 0.8194444444444444}
Azure Evaluation: {'Levenshtein Distance': 74, 'Word Error Rate': 0.7777777777777778, 'Character Error Rate': 0.2777777777777778}


In [ ]:
h, w,c = image.shape
boxes = pytesseract.image_to_boxes(image_thresh)

# Draw rectangles around detected text
for b in boxes.splitlines():
    b = b.split(' ')
    x1, y1, x2, y2 = int(b[1]), int(b[2]), int(b[3]), int(b[4])

    # Fix y-coordinates by subtracting from image height
    y1, y2 = h - y1, h - y2

    # Draw the bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display the image
cv2.imshow('img', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


2025-03-18 15:17:29.048 Python[23680:4222234] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-18 15:17:29.048 Python[23680:4222234] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 

In [ ]:
import xml.etree.ElementTree as ET

# Load and parse the XML file
xml_file = "boxes2.xml" 
tree = ET.parse(xml_file)
root = tree.getroot()

# Extract bounding boxes
ground_truth_boxes = []
for obj in root.findall("object"):
    name = obj.find("name").text
    bndbox = obj.find("bndbox")
    
    bbox = {
        "name": name,
        "xmin": int(bndbox.find("xmin").text),
        "ymin": int(bndbox.find("ymin").text),
        "xmax": int(bndbox.find("xmax").text),
        "ymax": int(bndbox.find("ymax").text),
    }
    
    ground_truth_boxes.append(bbox)

print(ground_truth_boxes)


[{'name': 'Succes in rezolvarea', 'xmin': 74, 'ymin': 291, 'xmax': 1340, 'ymax': 465}, {'name': 'temelor la', 'xmin': 127, 'ymin': 573, 'xmax': 1049, 'ymax': 731}, {'name': 'laboratoarele de', 'xmin': 73, 'ymin': 920, 'xmax': 1004, 'ymax': 1036}, {'name': 'inteligenta artificiala', 'xmin': 88, 'ymin': 1120, 'xmax': 1459, 'ymax': 1377}]


In [ ]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the intersection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [ ]:
# Perform OCR
with open(image_path, 'rb') as image_file:
        ocr_result = computervision_client.read_in_stream(
        image=image_file,mode="Printed",raw=True)

        operation_location = ocr_result.headers["Operation-Location"]
        operation_id = operation_location.split("/")[-1]


        while True:
            result = computervision_client.get_read_result(operation_id)
            ground_truth_box = [];
            if result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)
        result_box = []
        if result.status == OperationStatusCodes.succeeded:
            read_results = result.analyze_result.read_results
            h, w, c = image.shape
            i=0;
            for read_result in read_results:
                for line in read_result.lines:
                    for word in line.words:
                        bbox = word.bounding_box
                        result_box.append(bbox)
                        x1, y1 = int(bbox[0]), int(bbox[1])
                        x2, y2 = int(bbox[4]), int(bbox[5])
                        image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        for box in ground_truth_boxes:
            cv2.rectangle(image, (box["xmin"], box["ymin"]), (box["xmax"], box["ymax"]), (0, 0, 255), 2)  # Blue for GT
        for i in range(len(ground_truth_box)):
            bbox=result_box[i]
            box = ground_truth_boxes[i]
            coordonate_result = [bbox[0],bbox[1],bbox[4],bbox[5]]
            coordonate_ground = [box["xmin"], box["ymin"],box["xmax"],box["ymax"]]
            iou = bb_intersection_over_union(coordonate_result,coordonate_ground)
            print(f"IOU of{box['name']} is {iou}")
        # Display the image
        cv2.imshow('img', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()